# Análisis de texto en Python: Preprocesamiento

* * * 

<div class="alert alert-success"> 

### Objetivos de aprendizaje

* Aprender los pasos comunes para el preprocesamiento de datos de texto, así como las operaciones específicas para el preprocesamiento de datos de Twitter.
* Conocer los paquetes de PLN más utilizados y sus capacidades.
* Comprender los tokenizadores y cómo han cambiado desde la llegada de los Modelos de Lenguaje Grandes.
</div>

### Iconos utilizados en este cuaderno
🔔 **Pregunta**: Una pregunta rápida para ayudarte a entender qué está pasando.
🥊 **Desafío**: Ejercicios interactivos. ¡Los trabajaremos en el taller!<br>
⚠️ **Advertencia:** Tenga cuidado con cosas complicadas o errores comunes.<br>
🎬 **Demostración**: Mostrando algo más avanzado, ¡para que sepas para qué se puede usar Python!<br>

### Secciones
1. [Preprocesamiento](#section1)
2. [Tokenizacion](#section2)

En esta serie de talleres de tres partes, aprenderemos los fundamentos para realizar análisis de texto en Python. Estas técnicas se enmarcan en el ámbito del Procesamiento del Lenguaje Natural (PLN). El PLN es un campo que se ocupa de la identificación y extracción de patrones del lenguaje, principalmente en textos escritos. A lo largo de la serie de talleres, interactuaremos con diversos paquetes para realizar análisis de texto: desde métodos de cadenas simples hasta paquetes específicos de PLN, como `nltk`, `spaCy` y los más recientes sobre Modelos de Lenguaje Grandes (`BERT`).

Ahora, instalemos estos paquetes correctamente antes de profundizar en el material.

In [ ]:
# Uncomment the following lines to install packages/model
# %pip install NLTK
# %pip install transformers
# %pip install spaCy
# !python -m spacy download en_core_web_sm

<a id='section1'></a>

# Preprocesamiento

En la Parte 1 de este taller, abordaremos el primer paso del análisis de texto. Nuestro objetivo es convertir los datos de texto sin procesar y desordenados a un formato consistente. Este proceso se conoce como **preprocesamiento**, **limpieza de texto** o **normalización de texto**.

Observará que, al finalizar el preprocesamiento, nuestros datos aún se encuentran en un formato legible y comprensible. En las Partes 2 y 3, comenzaremos a explorar la conversión de los datos de texto a una representación numérica, un formato que las computadoras pueden procesar con mayor facilidad.

🔔 **Pregunta**: Detengámonos un momento para reflexionar sobre **sus** experiencias previas trabajando con datos de texto.
- ¿Cuál es el formato de los datos de texto con los que han interactuado (texto plano, CSV o XML)?
- ¿De dónde provienen (corpus estructurado, datos extraídos de la web, datos de encuestas)?
- ¿Están desordenados (es decir, tienen un formato uniforme)?

## Procesos Comunes

El preprocesamiento no se logra con una sola línea de código. A menudo, comenzamos familiarizándonos con los datos y, a medida que avanzamos, obtenemos una comprensión más clara de la granularidad del preprocesamiento que queremos aplicar.

Normalmente, empezamos aplicando un conjunto de procesos comunes para limpiar los datos. Estas operaciones no alteran sustancialmente la forma ni el significado de los datos; sirven como un procedimiento estandarizado para remodelarlos en un formato consistente.

Los siguientes procesos, por ejemplo, se aplican comúnmente para preprocesar textos en inglés de diversos géneros. Estas operaciones se pueden realizar mediante funciones integradas de Python, como los métodos `string` y las expresiones regulares.

- Minúsculas
- Eliminar signos de puntuación
- Eliminar espacios en blanco adicionales
- Eliminar palabras vacías

Tras el procesamiento inicial, podemos optar por realizar procesos específicos para cada tarea, cuyos detalles suelen depender de la tarea posterior que queramos realizar y de la naturaleza de los datos textuales (es decir, sus características estilísticas y lingüísticas). Antes de profundizar en estas operaciones, ¡echemos un vistazo a nuestros datos!

### Importar los datos de texto

Los datos de texto con los que trabajaremos son un archivo CSV. Contiene tuits sobre aerolíneas estadounidenses, recopilados desde febrero de 2015.

Leamos el archivo `airline_tweets.csv` en un marco de datos con `pandas`.

In [ ]:
# Importar la librería pandas
import pandas as pd

# se define la ruta del archivo CSV que contiene los datos, ../ significa “sube un nivel de carpeta” (va a la carpeta padre).
csv_path = '../data/airline_tweets.csv'

# Usa Pandas (pd) para leer el archivo CSV y guardarlo en un DataFrame llamado tweets.
# sep=',' indica que el separador de las columnas en el CSV es la coma (,).
tweets = pd.read_csv(csv_path, sep=',')

In [ ]:
#Muestra las primeras 5 filas del DataFrame (por defecto son 5, pero se puede pasar otro número: tweets.head(10))
# Sirve para hacer una vista previa rápida de los datos cargados.
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


El dataframe tiene una fila por tuit. El texto del tuit se muestra en la columna `text`.
- `text` (`str`): el texto del tuit.

Otros metadatos de interés son:
- `airline_sentiment` (`str`): el sentimiento del tuit, etiquetado como "neutral", "positivo" o "negativo".
- `airline` (`str`): la aerolínea sobre la que se tuitea.
- `retweet count` (`int`): cuántas veces se retuiteó el tuit.

Echemos un vistazo a algunos de los tweets:

In [ ]:
# Accede a la columna text del DataFrame (donde están los tweets), he imprime los tweets por orden.
print(tweets['text'].iloc[0])
print(tweets['text'].iloc[1])
print(tweets['text'].iloc[2])

@VirginAmerica What @dhepburn said.
@VirginAmerica plus you've added commercials to the experience... tacky.
@VirginAmerica I didn't today... Must mean I need to take another trip!


🔔 **Pregunta**: ¿Qué has notado? ¿Cuáles son las características estilísticas de los tuits?

### Minúsculas

Si bien reconocemos que el uso de mayúsculas y minúsculas en una palabra es informativo, a menudo no trabajamos en contextos donde podamos utilizar esta información correctamente.

Con frecuencia, el análisis posterior que realizamos **no distingue entre mayúsculas y minúsculas**. Por ejemplo, en el análisis de frecuencia, queremos tener en cuenta las diversas formas de una misma palabra. Convertir los datos de texto en minúsculas facilita este proceso y simplifica nuestro análisis.

Podemos convertir fácilmente en minúsculas con el método de cadena [`.lower()`](https://docs.python.org/3/library/stdtypes.html#str.lower); consulte la [documentación](https://docs.python.org/3/library/stdtypes.html#string-methods) para obtener más funciones útiles.

Apliquémoslo al siguiente ejemplo:

In [ ]:
# Selecciona el tweet que está en la fila número 108, lo guarda en la variable "first_example"
# y lo imprime.
first_example = tweets['text'][108]
print(first_example)

@VirginAmerica I was scheduled for SFO 2 DAL flight 714 today. Changed to 24th due weather. Looks like flight still on?


In [ ]:
# Verifica si todo el texto de "first_example" está en minúscula.
print(first_example.islower())
print(f"{'=' * 50}")

# Imprime una línea separadora de 50 signos = para que el resultado sea más legible en la consola.
print(first_example.lower())
print(f"{'=' * 50}")

# Convierte todo el texto del tweet a minúsculas.
print(first_example.upper())

False
@virginamerica i was scheduled for sfo 2 dal flight 714 today. changed to 24th due weather. looks like flight still on?
@VIRGINAMERICA I WAS SCHEDULED FOR SFO 2 DAL FLIGHT 714 TODAY. CHANGED TO 24TH DUE WEATHER. LOOKS LIKE FLIGHT STILL ON?


### Eliminar espacios en blanco adicionales

A veces nos encontramos con textos con espacios en blanco innecesarios, como espacios, tabulaciones y caracteres de nueva línea, lo cual es particularmente común cuando el texto se extrae de páginas web. Antes de profundizar en los detalles, presentemos brevemente las expresiones regulares (regex) y el paquete `re`.

Las expresiones regulares son una forma eficaz de buscar patrones de cadenas específicos en corpus extensos. Su curva de aprendizaje es notablemente pronunciada, pero pueden ser muy eficientes una vez que las dominamos. Muchos paquetes de PLN dependen en gran medida de las expresiones regulares. Los evaluadores de expresiones regulares, como [regex101](https://regex101.com), son herramientas útiles tanto para comprender como para crear expresiones regulares.

Nuestro objetivo en este taller no es ofrecer una introducción profunda (ni siquiera superficial) a las expresiones regulares; en cambio, queremos presentarles para que estén mejor preparados para profundizar en el futuro.

El siguiente ejemplo es un poema de William Wordsworth. Como muchos poemas, el texto puede contener saltos de línea adicionales (es decir, caracteres de nueva línea, `\n`) que queremos eliminar.

In [ ]:
# Aquí se define la ruta del archivo que contiene el poema.
text_path = '../data/poem_wordsworth.txt'

# Abre el archivo en modo lectura, Se usa with porque garantiza que el archivo se cierre automáticamente al final, aunque ocurra un error.
#Lee todo el contenido del archivo y lo guarda en la variable text como un string.
with open(text_path, 'r') as file:
    text = file.read()
    file.close()

Como puedes ver, el poema está formateado como una cadena continua de texto con saltos de línea al final de cada línea, lo que dificulta su lectura.

In [7]:
text

"I wandered lonely as a cloud\n\n\nI wandered lonely as a cloud\nThat floats on high o'er vales and hills,\nWhen all at once I saw a crowd,\nA host, of golden daffodils;\nBeside the lake, beneath the trees,\nFluttering and dancing in the breeze.\n\nContinuous as the stars that shine\nAnd twinkle on the milky way,\nThey stretched in never-ending line\nAlong the margin of a bay:\nTen thousand saw I at a glance,\nTossing their heads in sprightly dance.\n\nThe waves beside them danced; but they\nOut-did the sparkling waves in glee:\nA poet could not but be gay,\nIn such a jocund company:\nI gazed—and gazed—but little thought\nWhat wealth the show to me had brought:\n\nFor oft, when on my couch I lie\nIn vacant or in pensive mood,\nThey flash upon that inward eye\nWhich is the bliss of solitude;\nAnd then my heart with pleasure fills,\nAnd dances with the daffodils."

Una función útil para mostrar el poema correctamente es `.splitlines()`. Como su nombre indica, divide una secuencia de texto larga en una lista de líneas cuando hay un carácter de nueva línea.

In [ ]:
# Divide el contenido del poema (text) en líneas y devuelve una lista.
text.splitlines()

['I wandered lonely as a cloud',
 '',
 '',
 'I wandered lonely as a cloud',
 "That floats on high o'er vales and hills,",
 'When all at once I saw a crowd,',
 'A host, of golden daffodils;',
 'Beside the lake, beneath the trees,',
 'Fluttering and dancing in the breeze.',
 '',
 'Continuous as the stars that shine',
 'And twinkle on the milky way,',
 'They stretched in never-ending line',
 'Along the margin of a bay:',
 'Ten thousand saw I at a glance,',
 'Tossing their heads in sprightly dance.',
 '',
 'The waves beside them danced; but they',
 'Out-did the sparkling waves in glee:',
 'A poet could not but be gay,',
 'In such a jocund company:',
 'I gazed—and gazed—but little thought',
 'What wealth the show to me had brought:',
 '',
 'For oft, when on my couch I lie',
 'In vacant or in pensive mood,',
 'They flash upon that inward eye',
 'Which is the bliss of solitude;',
 'And then my heart with pleasure fills,',
 'And dances with the daffodils.']

Volvamos a nuestros datos de tweets para ver un ejemplo.

In [ ]:
# Selecciona el tweet número 5 de la columna text en el DataFrame tweets y lo guarda en la variable second_example y muestra el contenido.
second_example = tweets['text'][5]
second_example

"@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA"

En este caso, no queremos dividir el tuit en una lista de cadenas. Seguimos esperando una sola cadena de texto, pero queremos eliminar por completo el salto de línea.

El método de cadena `.strip()` elimina eficazmente los espacios en ambos extremos del texto. Sin embargo, no funcionará en nuestro ejemplo, ya que el carácter de nueva línea está en medio de la cadena.

In [ ]:
# El método .strip() elimina espacios en blanco al inicio y al final del tweet.
second_example.strip()

"@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.\nit's really the only bad thing about flying VA"

Aquí es donde las expresiones regulares pueden ser realmente útiles.

In [1]:
import re

Ahora, con expresiones regulares, básicamente las llamamos para que coincidan con un patrón identificado en los datos de texto, y queremos realizar algunas operaciones con la parte coincidente: extraerla, reemplazarla o eliminarla por completo. Por lo tanto, el funcionamiento de las expresiones regulares se puede resumir en los siguientes pasos:

- Identificar y escribir el patrón en la expresión regular (`r'PATTERN'`)
- Escribir el reemplazo para el patrón (`'REPLACEMENT'`)
- Llamar a la función específica de la expresión regular (p. ej., `re.sub()`)

En nuestro ejemplo, el patrón que buscamos es `\s`, que es la abreviatura de la expresión regular para cualquier espacio en blanco (`\n` y `\t` incluidos). También añadimos un cuantificador `+` al final: `\s+`. Esto significa que queremos capturar una o más ocurrencias del espacio en blanco.

In [ ]:
# Se prepara una expresión regular para limpiar espacios:
#\s+ → significa “uno o más espacios en blanco” (incluye tabulaciones y saltos de línea).
blankspace_pattern = r'\s+'

El reemplazo de uno o más espacios en blanco es exactamente un solo espacio, que es el límite canónico de palabras en inglés. Cualquier espacio adicional se reducirá a un solo espacio.

In [ ]:
#blankspace_repl = ' ' → dice que cada grupo de espacios será reemplazado por un solo espacio.
blankspace_repl = ' '

Finalmente, combinemos todo usando la función [`re.sub()`](https://docs.python.org/3.11/library/re.html#re.sub), lo que significa que queremos sustituir un patrón por un reemplazo. La función acepta tres argumentos: el patrón, el reemplazo y la cadena a la que queremos aplicar la función.

In [ ]:
# re.sub() busca en second_example todos los espacios múltiples (\s+) y los reemplaza por un único espacio.
#Así se normaliza el texto y queda más limpio.
clean_text = re.sub(pattern = blankspace_pattern, 
                    repl = blankspace_repl, 
                    string = second_example)
print(clean_text)

@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing. it's really the only bad thing about flying VA


¡Ta-da! El carácter de nueva línea ya no está.

### Eliminar signos de puntuación

A veces solo nos interesa analizar **caracteres alfanuméricos** (es decir, letras y números), en cuyo caso podríamos querer eliminar los signos de puntuación.

El módulo `string` contiene una lista de signos de puntuación predefinidos. Vamos a imprimirlos.

In [ ]:
# Importa la variable punctuation desde el módulo string.
# Esta variable contiene todos los signos de puntuación comunes en inglés
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


En la práctica, para eliminar estos caracteres de puntuación, podemos simplemente iterar sobre el texto y eliminar los caracteres que se encuentran en la lista, como se muestra a continuación en la función `remove_punct`.

In [ ]:
#Define una función llamada remove_punct para eliminar signos de puntuación en cualquier texto.
def remove_punct(text):
    '''Remove punctuation marks in input text'''
    
    # Si el carácter NO está en la lista de punctuation, lo guarda en la lista no_punct.
    no_punct = []
    for char in text:
        if char not in punctuation:
            no_punct.append(char)

    # Une todos los caracteres filtrados y devuelve el texto sin puntuación.
    text_no_punct = ''.join(no_punct)   
    
    return text_no_punct

Apliquemos la función al ejemplo siguiente.

In [ ]:
# Selecciona el tweet número 20, lo imprime y dibuja una línea separadora con "=" * 50
third_example = tweets['text'][20]
print(third_example)
print(f"{'=' * 50}")

# Llama a la función remove_punct() para eliminar todos los signos de puntuación del tweet 20.
remove_punct(third_example)

@VirginAmerica why are your first fares in May over three times more than other carriers when all seats are available to select???


'VirginAmerica why are your first fares in May over three times more than other carriers when all seats are available to select'

Vamos a intentarlo con otro tuit. ¿Qué has notado?

In [ ]:
# Imprime el tweet número 100 del dataset.
print(tweets['text'][100])
print(f"{'=' * 50}")

# versión del tweet sin signos de puntuación.
remove_punct(tweets['text'][100])

@VirginAmerica trying to add my boy Prince to my ressie. SF this Thursday @VirginAmerica from LAX http://t.co/GsB2J3c4gM


'VirginAmerica trying to add my boy Prince to my ressie SF this Thursday VirginAmerica from LAX httptcoGsB2J3c4gM'

¿Qué tal el siguiente ejemplo?

In [ ]:
# Aquí se crea un string con: Contracciones: ("We've", "don't"); Símbolos: (?!?, #, @)
contraction_text = "We've got quite a bit of punctuation here, don't we?!? #Python @D-Lab."

# elimina puntuación tanto en datos reales (tweets) como en textos de prueba.
remove_punct(contraction_text)

'Weve got quite a bit of punctuation here dont we Python DLab'

⚠️ **Advertencia:** En muchos casos, queremos eliminar los signos de puntuación **después** de la tokenización, lo cual explicaremos en breve. Esto nos indica que el **orden** del preprocesamiento es crucial.

## 🥊 Desafío 1: Preprocesamiento con múltiples pasos

Hasta ahora hemos aprendido algunas operaciones de preprocesamiento. ¡Combinémoslas en una función! Esta función te resultará útil si trabajas con datos de texto en inglés confusos y quieres preprocesarlos con una sola función.

A continuación se muestra el ejemplo de datos de texto para el desafío 1. Escribe una función para:
- Convertir el texto en minúsculas
- Eliminar signos de puntuación
- Eliminar espacios en blanco adicionales

¡Siéntete libre de reciclar los códigos que usamos anteriormente!

In [ ]:
# Abre el archivo (example1.txt), guarda el contenido en la variable challenge1.
#Imprime en pantalla para ver cómo luce antes de limpiarlo.
challenge1_path = '../data/example1.txt'

with open(challenge1_path, 'r') as file:
    challenge1 = file.read()
    
print(challenge1)



This is a text file that has some extra blankspace at the start and end. Blankspace is a catch-all term for spaces, tabs, newlines, and a bunch of other things that computers distinguish but to us all look like spaces, tabs and newlines.


The Python method called "strip" only catches blankspace at the start and end of a string. But it won't catch it in       the middle,		for example,

in this sentence.		Once again, regular expressions will

help		us    with this.





In [ ]:
#
def clean_text(text):

    # Convertir a minúsculas
    text = ...

    # Usa la función que ya definiste antes (remove_punct) para eliminar comas, puntos, etc.
    text = ...

    # Eliminar espacios en blanco adicionales
    text = ...

    return text

In [ ]:
# versión limpia del archivo (example1.txt)
# clean_text(challenge1)

## Procesos específicos de cada tarea

Ahora que comprendemos las operaciones comunes de preprocesamiento, aún quedan algunas operaciones adicionales por considerar. Nuestros datos de texto podrían requerir una mayor normalización según el idioma, la fuente y el contenido de los datos.

Por ejemplo, si trabajamos con documentos financieros, podríamos querer estandarizar los símbolos monetarios convirtiéndolos en dígitos. En nuestros datos de tuits, existen numerosos hashtags y URL. Estos se pueden reemplazar con marcadores de posición para simplificar el análisis posterior.

### 🎬 **Demostración**: Eliminar Hashtags y URL

Aunque las URL, los hashtags y los números son informativos por sí mismos, a menudo no nos importa su significado exacto.

Si bien podríamos eliminarlos por completo, suele ser informativo saber que **existe** una URL o un hashtag. En la práctica, reemplazamos las URL y los hashtags individuales con un "símbolo" que preserva la existencia de estas estructuras en el texto. Lo habitual es usar las cadenas "URL" y "HASHTAG".

Dado que estos tipos de texto suelen seguir una estructura regular, son un buen ejemplo para usar expresiones regulares. Apliquemos estos patrones a los datos de los tuits.

In [ ]:
# Se obtiene el tweet número 13 del dataset tweets y lo imprime.
url_tweet = tweets['text'][13]
print(url_tweet)

@VirginAmerica @virginmedia I'm flying your #fabulous #Seductive skies again! U take all the #stress away from travel http://t.co/ahlXHhKiyn


In [ ]:
# (url_pattern) es una expresión regular que detecta enlaces.
#(http|ftp|https):\/\/ → busca enlaces que empiecen con http, https o ftp.
#([\w_-]+(?:(?:\.[\w_-]+)+)) → busca dominios (ej. twitter.com)
# ([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-]) → busca la parte extra después del dominio (parámetros, rutas, etc).
url_pattern = r'(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])'
url_repl = ' URL '
# re.sub(...) reemplaza todo el enlace completo con la palabra " URL ".
re.sub(url_pattern, url_repl, url_tweet)

"@VirginAmerica @virginmedia I'm flying your #fabulous #Seductive skies again! U take all the #stress away from travel  URL "

In [ ]:
# Reemplazar hashtags por "HASHTAG".
hashtag_pattern = r'(?:^|\s)[＃#]{1}(\w+)'
hashtag_repl = ' HASHTAG '
re.sub(hashtag_pattern, hashtag_repl, url_tweet)

"@VirginAmerica @virginmedia I'm flying your HASHTAG  HASHTAG  skies again! U take all the HASHTAG  away from travel http://t.co/ahlXHhKiyn"

<a id='section2'></a>

# Tokenización

## Tokenizadores antes de los LLM

Uno de los pasos más importantes en el análisis de texto es la tokenización. Este proceso consiste en descomponer una secuencia larga de texto en tokens de palabras. Con estos tokens disponibles, estamos listos para realizar un análisis a nivel de palabra. Por ejemplo, podemos filtrar los tokens que no contribuyen al significado central del texto.

En esta sección, presentaremos cómo realizar la tokenización utilizando `nltk`, `spaCy` y un Modelo de Lenguaje Grande (`bert`). El objetivo es presentarle diferentes paquetes de PLN, ayudarle a comprender sus funcionalidades y mostrarle cómo acceder a las funciones clave de cada paquete.

### `nltk`

El primer paquete que usaremos se llama **Natural Language Toolkit**, o `nltk`.

Instalemos un par de módulos del paquete.

In [26]:
import nltk

In [ ]:
# wordnet → recurso léxico en inglés (se usa para sinónimos, lematización, etc).
# nltk.download('wordnet')
#stopwords → lista de palabras vacías (ej: "the", "and", "is", que no aportan mucho significado).
# nltk.download('stopwords')
#punkt → modelo de tokenización pre-entrenado (sirve para cortar oraciones y palabras).
# nltk.download('punkt')

`nltk` tiene una función llamada `word_tokenize`. Requiere un argumento, que es el texto que se tokenizará, y nos devuelve una lista de tokens.

In [ ]:
# Cargar word_tokenize
from nltk.tokenize import word_tokenize

# Toma el tweet número 7 del dataset.
text = tweets['text'][7]
print(text)

@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP


In [ ]:
# word_tokenize(text) divide el tweet en tokens (palabras, números, signos de puntuación, etc).
nltk_tokens = word_tokenize(text)
nltk_tokens

['@',
 'VirginAmerica',
 'Really',
 'missed',
 'a',
 'prime',
 'opportunity',
 'for',
 'Men',
 'Without',
 'Hats',
 'parody',
 ',',
 'there',
 '.',
 'https',
 ':',
 '//t.co/mWpG7grEZP']

Aquí tenemos una lista de tokens identificados por `nltk`. ¡Revisémoslos un momento!

🔔 **Pregunta**: ¿Tienes sentido para ti los límites de palabras definidos por `nltk`? Presta atención al usuario de Twitter y a la URL del tuit de ejemplo.

Puede que te parezca que acceder a las funciones de `nltk` es bastante sencillo. La función que usamos anteriormente se importó del módulo `nltk.tokenize`, que, como su nombre indica, se encarga principalmente de la tokenización.

En esencia, `nltk` cuenta con [una colección de módulos](https://www.nltk.org/api/nltk.html) que cumplen diferentes funciones, por nombrar algunas:

| módulo NLTK   | Función                | Enlace                                                         |
|---------------|---------------------------|--------------------------------------------------------------|
| nltk.tokenize | Tokenización              | [Documentación](https://www.nltk.org/api/nltk.tokenize.html) |
| nltk.corpus   | Recuperar corpus integrados | [Documentación](https://www.nltk.org/nltk_data/)             |
| nltk.tag      | Etiquetado de partes del discurso    | [Documentación](https://www.nltk.org/api/nltk.tag.html)      |
| nltk.stem     | Derivado                  | [Documentación](https://www.nltk.org/api/nltk.stem.html)     |
| ...           | ...                       | ...                                                          |

Importemos `stopwords` desde el módulo `nltk.corpus`, que alberga una variedad de corpus integrados.

In [ ]:
from nltk.corpus import stopwords

Especifiquemos que queremos recuperar palabras vacías en inglés. La función simplemente devuelve una lista de palabras vacías, principalmente palabras de función, que `nltk` identifica.

In [ ]:
# # Carga la lista de palabras vacías en inglés.
#El [:10] solo imprime las primeras 10.
stop = stopwords.words('english')
stop[:10]

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an']

### `spaCy`
Además de `nltk`, contamos con otro paquete muy utilizado llamado `spaCy`. 

`spaCy` cuenta con su propia canalización de procesamiento. Recibe una cadena de texto, ejecuta la canalización `nlp` en ella y almacena el texto procesado y sus anotaciones en un objeto llamado `doc`. La canalización `nlp` siempre realiza la tokenización, así como otros componentes de análisis de texto solicitados por el usuario. Estos componentes son bastante similares a los módulos de `nltk`.

<img src='../images/spacy.png' alt="spacy pipeline" width="700">

Tenga en cuenta que siempre comenzamos inicializando la secuencia de comandos «nlp», según el idioma del texto. Aquí, cargamos un modelo de idioma preentrenado para inglés: «en_core_web_sm». El nombre sugiere que se trata de un modelo ligero entrenado con datos de texto (por ejemplo, blogs); consulte las descripciones de los modelos [aquí](https://spacy.io/models/en#en_core_web_sm).

Esta es la primera vez que nos encontramos con el concepto de preentrenamiento, aunque quizás ya lo haya oído en otros lugares. En el contexto del PLN, el preentrenamiento significa que el modelo se ha entrenado con una gran cantidad de datos. Como resultado, incorpora un cierto conocimiento de la estructura de las palabras y la gramática del idioma.

Por lo tanto, al aplicar el modelo a nuestros propios datos, podemos esperar que sea razonablemente preciso al realizar diversas tareas de anotación, por ejemplo, etiquetar la categoría gramatical de una palabra, identificar el núcleo sintáctico de una frase, etc.

¡Comencemos! Primero, debemos cargar el modelo de lenguaje preentrenado que instalamos anteriormente.

In [32]:
import spacy
nlp = spacy.load('en_core_web_sm')

La canalización `nlp` incluye, por defecto, un conjunto de componentes a los que podemos acceder mediante el atributo `.pipe_names`.

Puede que notes que no incluye el tokenizador. ¡No te preocupes! El tokenizador es un componente especial que la canalización siempre incluye.

In [33]:
# Retrieve components included in NLP pipeline
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

Ejecutemos el pipeline `nlp` en nuestros datos de tweet de ejemplo y asignémoslo a una variable `doc`.

In [34]:
# Apply the pipeline to example tweet
doc = nlp(tweets['text'][7])

En esencia, el objeto `doc` contiene los tokens (creados por el tokenizador) y sus anotaciones (creadas por otros componentes), que son características lingüísticas (https://spacy.io/usage/linguistic-features) útiles para el análisis de texto. Recuperamos el token y sus anotaciones accediendo a los atributos correspondientes. 

| Atributo     | Anotación                              | Enlace                                                                      |
|----------------|-----------------------------------------|---------------------------------------------------------------------------|
| token.text     | El token en texto textual              | [Documentation](https://spacy.io/api/token#attributes)                    |
| token.is_stop  | Si el token es una stop word        | [Documentation](https://spacy.io/api/attributes#_title)                   |
| token.is_punct | Si el token es un signo de puntuación | [Documentation](https://spacy.io/api/attributes#_title)                   |
| token.lemma_   | La forma base del token             | [Documentation](https://spacy.io/usage/linguistic-features#lemmatization) |
| token.pos_     | La etiqueta POS simple del token        | [Documentation](https://spacy.io/usage/linguistic-features#pos-tagging)   |
| ...            | ...                                     | ...                                                                       |

Primero, obtengamos los tokens. Iteraremos sobre el objeto `doc` y recuperaremos el texto de cada token.

In [35]:
# Get the verbatim texts of tokens
spacy_tokens = [token.text for token in doc]
spacy_tokens

['@VirginAmerica',
 'Really',
 'missed',
 'a',
 'prime',
 'opportunity',
 'for',
 'Men',
 'Without',
 'Hats',
 'parody',
 ',',
 'there',
 '.',
 'https://t.co/mWpG7grEZP']

In [36]:
# Get the NLTK tokens
nltk_tokens

['@',
 'VirginAmerica',
 'Really',
 'missed',
 'a',
 'prime',
 'opportunity',
 'for',
 'Men',
 'Without',
 'Hats',
 'parody',
 ',',
 'there',
 '.',
 'https',
 ':',
 '//t.co/mWpG7grEZP']

🔔 **Pregunta**: Detengámonos un momento para comparar los tokens generados por `nltk` y `spaCy`. ¿Qué han observado?

Recuerden que también podemos acceder a varias anotaciones de estos tokens. Por ejemplo, una anotación que ofrece `spaCy` es que codifica fácilmente si un token es una palabra vacía.

In [37]:
# Retrieve the is_stop annotation
spacy_stops = [token.is_stop for token in doc]

# The results are boolean values
spacy_stops

[False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False]

## 🥊 Desafío 2: Eliminar palabras vacías

Ya conocemos el funcionamiento de `nltk` y `spaCy` como paquetes de PLN. También hemos demostrado cómo identificar palabras vacías con cada paquete.

Escribamos **dos** funciones para eliminar palabras vacías de nuestros datos de texto.

- Completar la función para eliminar palabras vacías usando `nltk`
- El código inicial requiere dos argumentos: la entrada de texto sin formato y una lista de palabras vacías predefinidas.
- Completar la función para eliminar palabras vacías usando `spaCy`
- El código inicial requiere un argumento: la entrada de texto sin formato.

Un recordatorio antes de empezar: ambas funciones toman texto sin formato como entrada; ¡Eso es una señal para realizar primero la tokenización del texto sin formato!

In [ ]:
def remove_stopword_nltk(raw_text, stopword):
    pass
    
    # Step 1: Tokenization with nltk
    # YOUR CODE HERE
    
    # Step 2: Filter out tokens in the stop word list
    # YOUR CODE HERE

In [ ]:
def remove_stopword_spacy(raw_text):
    pass

    # Step 1: Apply the nlp pipeline
    # YOUR CODE HERE
    
    # Step 2: Filter out tokens that are stop words
    # YOUR CODE HERE

In [ ]:
# remove_stopword_nltk(text, stop)

In [ ]:
# remove_stopword_spacy(text)

## 🎬 **Demostración**: Potentes funciones de `spaCy`

El pipeline de procesamiento de lenguaje natural (PLN) de `spaCy` incluye diversas anotaciones lingüísticas que pueden resultar muy útiles para el análisis de texto.

Por ejemplo, podemos acceder a más anotaciones, como el lema, la etiqueta gramatical y su significado, y si el token se asemeja a una URL.

In [38]:
# Print tokens and their annotations
for token in doc:
    print(f"{token.text:<24} | {token.lemma_:<24} | {token.pos_:<12} | {spacy.explain(token.pos_):<12} | {token.like_url:<12} |")

@VirginAmerica           | @VirginAmerica           | PROPN        | proper noun  | 0            |
Really                   | really                   | ADV          | adverb       | 0            |
missed                   | miss                     | VERB         | verb         | 0            |
a                        | a                        | DET          | determiner   | 0            |
prime                    | prime                    | ADJ          | adjective    | 0            |
opportunity              | opportunity              | NOUN         | noun         | 0            |
for                      | for                      | ADP          | adposition   | 0            |
Men                      | Men                      | PROPN        | proper noun  | 0            |
Without                  | without                  | ADP          | adposition   | 0            |
Hats                     | Hats                     | PROPN        | proper noun  | 0            |
parody    

Como puedes imaginar, es habitual que este conjunto de datos contenga nombres de lugares y códigos de aeropuertos. Sería genial si pudiéramos identificarlos y extraerlos de los tuits.

In [39]:
# Print example tweets with place names and airport codes
tweet_city = tweets['text'][8273]
tweet_airport = tweets['text'][502]
print(tweet_city)
print(f"{'=' * 50}")
print(tweet_airport)

@JetBlue Vegas, San Francisco, Baltimore, San Diego and Philadelphia so far! I'm a very frequent business traveler.
@VirginAmerica Flying LAX to SFO and after looking at the awesome movie lineup I actually wish I was on a long haul.


Podemos utilizar el componente "ner" ([Reconocimiento de entidades nombradas](https://en.wikipedia.org/wiki/Named-entity_recognition)) para identificar entidades y sus categorías.

In [40]:
# Print entities identified from the text
doc_city = nlp(tweet_city)
for ent in doc_city.ents:
    print(f"{ent.text:<15} | {ent.start_char:<10} | {ent.end_char:<10} | {ent.label_:<10}")

Vegas           | 9          | 14         | GPE       
San Francisco   | 16         | 29         | GPE       
Baltimore       | 31         | 40         | GPE       
San Diego       | 42         | 51         | GPE       
Philadelphia    | 56         | 68         | GPE       


También podemos usar «desplazamiento» para resaltar las entidades identificadas en el texto y, al mismo tiempo, anotar la categoría de la entidad.

En el siguiente ejemplo, tenemos cuatro «GPE» (es decir, entidades geopolíticas, generalmente países y ciudades) identificadas.

In [41]:
# Visualize the identified entities
from spacy import displacy
displacy.render(doc_city, style='ent', jupyter=True)

Vamos a intentarlo con otro ejemplo.

In [42]:
# Print entities identified from the text
doc_airport = nlp(tweet_airport)
for ent in doc_airport.ents:
     print(f"{ent.text:<15} | {ent.start_char:<10} | {ent.end_char:<10} | {ent.label_:<10}")

@VirginAmerica  | 0          | 14         | CARDINAL  
Flying LAX      | 15         | 25         | ORG       
SFO             | 29         | 32         | ORG       


Es interesante que los códigos de los aeropuertos estén identificados como «ORG» (organizaciones) y el identificador del tweet como «CARDINAL».

In [43]:
# Visualize the identified entities
displacy.render(doc_airport, style='ent', jupyter=True)

## Tokenizadores desde los LLM

Hasta ahora, hemos visto cómo funciona la tokenización con dos paquetes de PLN ampliamente utilizados. Funcionan bastante bien en algunos entornos, pero no en otros. Recordemos que `nltk` tiene dificultades con las URL. Ahora, imaginemos que los datos que tenemos son aún más confusos, con errores ortográficos, palabras de reciente creación, nombres extranjeros, etc. (denominados colectivamente palabras "fuera de vocabulario" u OOV). En tales circunstancias, podríamos necesitar un modelo más potente para gestionar estas complejidades.

De hecho, los esquemas de tokenización cambian sustancialmente con los **Grandes Modelos de Lenguaje** (LLM), que son modelos entrenados con una enorme cantidad de datos de fuentes mixtas. Con esa magnitud de datos, los LLM son más eficaces para fragmentar una secuencia más larga en tokens y estos en **subtokens**. Estos subtokens pueden ser unidades morfológicas de una palabra, como un afijo, pero también pueden ser partes de una palabra donde el modelo establece un límite "significativo". En esta sección, demostraremos la tokenización en **BERT** (Representaciones de Codificador Bidireccional de Transformers), que utiliza un algoritmo de tokenización llamado [**WordPiece**](https://huggingface.co/learn/nlp-course/en/chapter6/6).

Cargaremos el tokenizador de BERT desde el paquete `transformers`, que aloja varios LLM basados ​​en Transformers (por ejemplo, BERT). No profundizaremos en la arquitectura de Transformer en este taller, pero pueden consultar el taller de D-lab sobre [Fundamentos de GPT](https://github.com/dlab-berkeley/GPT-Fundamentals).

Tokenización de WordPiece

Tenga en cuenta que BERT está disponible en varias versiones. La que exploraremos hoy es `bert-base-uncased`. Este modelo tiene un tamaño moderado (denominado `base`) y no distingue entre mayúsculas y minúsculas, lo que significa que el texto de entrada se escribirá en minúsculas por defecto.

In [ ]:
# Load BERT tokenizer in
from transformers import BertTokenizer

# Initialize the tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

El tokenizador tiene múltiples funciones, como veremos en un minuto. Ahora queremos acceder a la función `.tokenize()` desde el tokenizador.

Tokenicemos un tweet de ejemplo a continuación. ¿Qué has notado?

In [45]:
# Select an example tweet from dataframe
text = tweets['text'][194]
print(f"Text: {text}")
print(f"{'=' * 50}")

# Apply tokenizer
tokens = tokenizer.tokenize(text)
print(f"Tokens: {tokens}")
print(f"Number of tokens: {len(tokens)}")

Text: @VirginAmerica Just DM'd. Same issue persisting.
Tokens: ['@', 'virgin', '##ame', '##rica', 'just', 'd', '##m', "'", 'd', '.', 'same', 'issue', 'persist', '##ing', '.']
Number of tokens: 15


Los símbolos de doble "hashtag" (`##`) hacen referencia a un token de subpalabra: un segmento separado del token anterior.

🔔 **Pregunta**: ¿Tienen sentido estas subpalabras?

Un avance significativo de los LLM es que a cada token se le asigna un ID de su vocabulario. Nuestra computadora no entiende el texto en su forma original, por lo que cada token se traduce en un ID. Estos ID son las entradas a las que el modelo accede y con las que opera.

Los tokens y los ID se pueden convertir bidireccionalmente, por ejemplo:

In [46]:
# Get the input ID of the word 
print(f"ID of just is: {tokenizer.vocab['just']}")

# Get the text of the input ID
print(f"Token 2074 is: {tokenizer.decode([2074])}")

ID of just is: 2074
Token 2074 is: just


Convirtamos los tokens en identificaciones de entrada.

In [47]:
# Convert a list of tokens to a list of input IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"Number of input IDs: {len(input_ids)}")
print(f"Input IDs of text: {input_ids}")

Number of input IDs: 15
Input IDs of text: [1030, 6261, 14074, 14735, 2074, 1040, 2213, 1005, 1040, 1012, 2168, 3277, 29486, 2075, 1012]


### Tokens Especiales

Además de los tokens y subtokens mencionados anteriormente, BERT también utiliza tres tokens especiales: `SEP`, `CLS` y `UNK`. El token `SEP` actúa como terminador de oración, comúnmente conocido como token `EOS` (Fin de Oración). El token `UNK` representa cualquier token que no se encuentre en el vocabulario, de ahí los tokens "desconocidos". El token `CLS` se añade al principio de la oración. Su origen se remonta a tareas de clasificación de texto (p. ej., detección de spam), donde los investigadores encontraron útil un token que agregara la información de toda la oración para fines de clasificación.

Cuando aplicamos `tokenizer()` directamente a nuestros datos de texto, le pedimos a BERT que **codifique** el texto. Esto implica varios pasos:
- Tokenizar el texto
- Añadir tokens especiales
- Convertir tokens en ID de entrada
- Otros procesos específicos del modelo

Vamos a imprimirlos.

In [48]:
# Get the input IDs by providing the key 
input_ids_from_tokenizer = tokenizer(text)['input_ids']
print(f"Number of input IDs: {len(input_ids_from_tokenizer)}")
print(f"IDs from tokenizer: {input_ids_from_tokenizer}")

Number of input IDs: 17
IDs from tokenizer: [101, 1030, 6261, 14074, 14735, 2074, 1040, 2213, 1005, 1040, 1012, 2168, 3277, 29486, 2075, 1012, 102]


Parece que hemos añadido dos tokens más: 101 y 102.

¡Vamos a convertirlos en texto!

In [49]:
# Convert input IDs to texts
print(f"The 101st token: {tokenizer.convert_ids_to_tokens(101)}")
print(f"The 102nd token: {tokenizer.convert_ids_to_tokens(102)}")

The 101st token: [CLS]
The 102nd token: [SEP]


Como puede ver, nuestro ejemplo de texto ahora es una lista de identificadores de vocabulario. Además, BERT añade el terminador de oración «SEP» y el token inicial «CLS» al texto original. El tokenizador de BERT también codifica una gran cantidad de textos; posteriormente, están listos para su posterior procesamiento.

## 🥊 Desafío 3: Encuentra el límite de palabra

Ahora que sabemos que la tokenización en BERT suele generar subpalabras, probemos con algunos ejemplos más.

- ¿Cuál crees que es el límite correcto para dividir las siguientes palabras en subpalabras?
- ¿Qué otros ejemplos has probado?

In [50]:
def get_tokens(string):
    '''Tokenzie the input string with BERT'''
    tokens = tokenizer.tokenize(string)
    return print(tokens)

In [51]:
# Abbreviations
get_tokens('dlab')

# OOV
get_tokens('covid')

# Prefix
get_tokens('huggable')

# Digits
get_tokens('378')

# YOUR EXAMPLE

['dl', '##ab']
['co', '##vid']
['hug', '##ga', '##ble']
['37', '##8']


Concluiremos la Parte 1 con este desafío (que esperamos) que invita a la reflexión. Los LLM suelen incluir un esquema de tokenización mucho más sofisticado, pero existe un debate continuo sobre sus limitaciones en aplicaciones prácticas. La sección de referencia incluye algunas entradas de blog que abordan este problema. ¡No dudes en explorar más a fondo si esta pregunta te parece interesante!

## Referencias

1. Un tutorial que presenta el esquema de tokenización en BERT: [El curso de PNL de huggingface sobre tokenización de piezas de palabra](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt)
2. Un ejemplo específico de "fracaso" en la tokenización: [Debilidades de la tokenización de piezas de palabra: Hallazgos desde la primera línea del PNL en VMware](https://medium.com/@rickbattle/weaknesses-of-wordpiece-tokenization-eb20e37fec99)
3. ¿Cómo determina BERT los límites entre subtokens? [Tokenización de subpalabras en BERT](https://tinkerd.net/blog/machine-learning/bert-tokenization/#subword-tokenization)

<div class="alert alert-success">

## ❗ Puntos clave

* El preprocesamiento incluye varios pasos, algunos más comunes para los datos de texto y otros específicos de cada tarea.
* Tanto `nltk` como `spaCy` pueden utilizarse para la tokenización y la eliminación de palabras vacías. Este último es más potente a la hora de proporcionar diversas anotaciones lingüísticas.
* La tokenización funciona de forma diferente en BERT, que a menudo implica la descomposición de una palabra completa en subpalabras.

</div>